In [24]:
import jupyter
from tweet_browser_test import tweet_browser as tb
import ipywidgets as widgets
import voila
from matplotlib import pyplot as plt
from ipysheet import sheet, cell, row, column, cell_range
from IPython.display import display


colHeaders = ["SenderScreenName", "CreatedTime", "Message"]   
operators = [None] * 4
#currentOperator = 0
s = tb.createSession("allCensus_sample.csv")
#output = []
out = widgets.Output()

def selectColumns (row, colHeaders: list):
    result = []
    for j in colHeaders:
        result.append(row[tb.headerDict[j]])
    return result

def resetDisplay():
    global operator
    out.clear_output()
    with out:
        display(grouped_buttons)
        display(operator)
def clear(b):
    resetDisplay()

def randomSample(b):
    if s.currentSet != s.base:
        s.back()
    s.simpleRandomSample(sampleSelector.value)
    getTweets(b)

def weightedSample(b):
    if s.currentSet != s.base:
        s.back()
    s.weightedSample(sampleSelector.value, weightedSelector.value)
    getTweets(b)
    
def searchKeyword(b):
    if s.currentSet != s.base:
        s.back()
    if searchType.value == 'Contains Any':
        s.searchKeyword(searchBox.value.split(' '), True)
    elif searchType.value == 'Contains All':
        s.searchKeyword(searchBox.value.split(' '))
    elif searchType.value == 'Advanced':
        s.advancedSearch(searchBox.value)
    elif searchType.value == 'Regular Expression':
        s.regexSearch(searchBox.value)
    else:
        return
    getTweets(b)

def filterBy(b):
    if s.currentSet != s.base:
        s.back()
    index = filterBox.value.find(" ")
    if index == -1:
        print("Invalid input")
        return
    columnName = filterBox.value[:index]
    if not (columnName in tb.headerDict):
        print("Invalid column")
        return
    value = filterBox.value[index + 1:]
    s.filterBy(columnName, value)
    getTweets(b)
    
def getTweets(b):
    resetDisplay()
    if s.currentSet.size > 1000:
        with out:
            print("Table too large to display")
        return
    if s.currentSet.size == 0:
        with out:
            print("No rows selected")
    output = s.getCurrentSubset()
    sheet1 = sheet(rows=s.currentSet.size, columns = len(colHeaders), column_width = 100, column_headers = ["SenderScreenName", "CreatedTime", "Message"])
    rowNum = 0
    #newOut = [] 
    for i in output:
        rowi = row(rowNum, selectColumns(i, colHeaders))
        rowNum += 1
        #newOut.append(selectColumns(i, colHeaders)) 
    with out:
        #plt.table(newOut) 
        #plt.axis("off")
        #display(plt.show()) 
        display(sheet1)

####### UI Widgets #######

# Selection Dropdown
selector = widgets.Dropdown(
    options=['Random Sample', 'Weighted Sample', 'Search Keyword', 'Filter By'],
    value='Random Sample', description='Operation:')
clearButton = widgets.Button(description='Clear Tweets', tooltip='Clear current Tweets')
grouped_buttons = widgets.HBox([selector, clearButton])
clearButton.on_click(clear)

# Random Sample
randomSampleButton = widgets.Button(description='Get Tweets', tooltip='Get Tweets')   
sampleSelector = widgets.IntSlider(value=300, min=0, max=s.currentSet.size, step=1, description='Sample size',
    continuous_update=False, orientation='horizontal', readout=True, readout_format='d')
randomSampleBar = widgets.HBox([sampleSelector, randomSampleButton])
randomSampleButton.on_click(randomSample)
operators[0] = randomSampleBar

# Weighted Sample
weightedButton = widgets.Button(description='Get Tweets', tooltip='Get Tweets')  
weightedSelector = widgets.Text(value='', placeholder='<Column_name>', description='Weight by')
weightedBar = widgets.HBox([sampleSelector, weightedSelector, weightedButton])
weightedButton.on_click(weightedSample)
operators[1] = weightedBar

#Search Bar
searchBox = widgets.Text(value='', placeholder='Keyword', description='Input:')
searchButton = widgets.Button(description='Search', tooltip='Run search on current input')
ContainsAnyToolTip = 'Returns tweets with any typed keyword (seperated by spaces)'
ContainsAllToolTip = 'Returns tweets containing all typed space-separated keywords'
AdvancedSearchToolTip = 'Search for tweets that match the inputted logical expression'
RegularExpressionToolTip = 'Returns tweets that match regular expression'
searchType = widgets.ToggleButtons(options=['Contains Any', 'Contains All', 'Advanced', 'Regular Expression'],
    tooltips=[ContainsAnyToolTip, ContainsAllToolTip, AdvancedSearchToolTip, RegularExpressionToolTip])
searchBar = widgets.VBox([widgets.HBox([searchBox, searchButton]), searchType])
searchButton.on_click(searchKeyword)
operators[2] = searchBar

#Filer By
filterBox = widgets.Text(value='', placeholder='<Column_name> <value>', description='Filter:')
filterButton = widgets.Button(description='Filter', tooltip='Filty by column on current input')
filterBar = widgets.HBox([filterBox, filterButton])
filterButton.on_click(filterBy)
operators[3] = filterBar

#HTML Table (test)
#table = widgets.HTML(value="Hello <b>World</b>")
#display(table)

operator = operators[0]
with out:
    display(grouped_buttons)
    display(operator)
#operator.layout.visibility = 'hidden'

def selctionHandler(change):
    global operator
    operator = operators[change['owner'].index]
    resetDisplay()
    
selector.observe(selctionHandler, names='value')
out

HTML(value='Hello <b>World</b>')

Output()